In [ ]:
from PIL import Image
from zipfile import ZipFile
from PIL import Image
import pytesseract
import cv2 as cv
from PIL import ImageDraw
import numpy as np
import PIL
import math

name=input()
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
with ZipFile('small_img.zip', 'r') as zip:
    for item in (zip.infolist()):
        with zip.open(item) as pict:
            img = Image.open(pict)
            img = img.convert('L')
            im = np.array(img)
            text = pytesseract.image_to_string(img)
            text = list(text.split())
            images=[]
            if name in text:
                faces = face_cascade.detectMultiScale(im, 1.3,5)
                pil_img = Image.open(pict).convert("RGB")
                drawing = ImageDraw.Draw(pil_img)
                for x, y, w, h in faces:
                    drawing.rectangle((x, y, x+w, y+h), outline="white")
                    bounding_box=(x,y,x+w,y+h)
                    title_image=pil_img.crop(bounding_box)
                    images.append(title_image)
                no_img=len(images)
                avg_height=0
                avg_width=0
                first=images[0]
                print("Results found in file {}".format(item.filename))
                if len(images)==0:
                    print("But there were no faces in that file")
                for pic in images:
                    avg_width+=pic.width
                    avg_height+=pic.height    
                avg_height=int(avg_height/no_img)
                avg_width=int(avg_width/no_img)    
                contact_sheet=PIL.Image.new(first.mode,(avg_width*5,avg_height*(math.ceil(no_img/5))))
                x=0
                y=0
                for i in images:
                    i=i.resize((avg_width,avg_height))
                    contact_sheet.paste(i, (x, y) )
                    if x+avg_width == contact_sheet.width:
                        x=0
                        y=y+avg_height
                    else:
                        x=x+avg_width
                contact_sheet = contact_sheet.resize((int(contact_sheet.width/2),int(contact_sheet.height/2) ))    
                display(contact_sheet)
